In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-13"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8433,2024-05-13,Eua Nba,20:00,Cleveland Cavaliers,Boston Celtics,3.67,1.31,207.5,1.80,1.95,6.5,2.05,1.08,vcMXAHxR,0.272480,0.763359,0.555556,0.512821,0.035838,0.8,0.2,NaN,NaN,221.336,78.571467,0.354987,150.016,20.449877,0.136318,208.020,155.544,351.54,137.80,1.0,0.0,0.0,0.0,0.670190,0.056569,0.438271,-0.30,-0.060,-44.500000,0.604518,0.6,-0.004518,-1.01,-0.202,-1.534653,0.693725,0.8,0.106275
8434,2024-05-13,Eua Nba,22:30,Dallas Mavericks,Oklahoma City Thunder,1.85,2.02,214.5,1.80,1.95,-2.5,2.00,2.30,Io2yKiA5,0.540541,0.495050,0.555556,0.512821,0.035590,0.2,0.4,NaN,NaN,181.574,32.952212,0.181481,191.066,30.669919,0.160520,178.092,222.122,174.30,233.31,0.0,0.0,0.0,0.0,0.062123,0.056569,0.098666,3.34,0.668,1.272455,0.672556,1.0,0.327444,0.57,0.114,8.947368,0.533459,0.6,0.066541
8435,2024-05-13,Colômbia Lbp,21:40,Corsarios Cartagena,Cimarrones del Choco,1.83,1.89,145.5,1.85,1.85,-1.5,1.97,2.05,U74C8mrf,0.546448,0.529101,0.540541,0.540541,0.075549,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.022810,0.000000,0.028144,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8436,2024-05-13,Estônia Korvpalli Meistriliiga,13:00,Pärnu,Tartu Ulikool,2.29,1.58,153.5,1.85,1.85,2.5,1.97,1.63,hhEH4aOp,0.436681,0.632911,0.540541,0.540541,0.069593,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,159.90,146.20,0.0,0.0,0.0,0.0,0.259455,0.000000,0.133565,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,22:30,Eua Nba,Dallas Mavericks,Oklahoma City Thunder,214.5,1.80,1.0,Over
1,21:40,Colômbia Lbp,Corsarios Cartagena,Cimarrones del Choco,145.5,1.85,1.0,Over
